<a href="https://colab.research.google.com/github/moosemaniam/IISCDeepLearning/blob/main/M1_AST_09_Scalable_Programming_using_PySpark_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Programme in Deep Learning (Foundations and Applications)
## A Program by IISc and TalentSprint
### Assignment 9: PySpark Transformation on Paired RDD's

## Learning Objectives

At the end of the experiment, you will be able to

* perform paired RDD (Resilient Distributed Datasets) operations like transformations.

* know the ways to transform the paired RDD's with by applying tranformations and action on it.

## Information

### Paired RDD's
Pair-wise RDD's are RDD's in which each element is in the form of tuples where the first element is key and the second element is the value.
* They have the keys and values associated. 
* Here keys are not distinct and a single value is assigned for a value.
* They are used to perform aggregate operations. 
* Paired RDD's exposes additional transformation and actions.
* They are derived from base RDD. 
* It supports all base operations.



### Setup Steps:

In [5]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2200009" #@param {type:"string"}

In [6]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "538477474" #@param {type:"string"}

In [7]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()
  
notebook= "M1_AST_09_Scalable_Programming_using_PySpark_A" #name of the notebook
def setup():
#  ipython.magic("sx pip3 install torch")  
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None
    
    elif getAnswer1() and getAnswer2() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer1" : Answer1, "answer2" : Answer2, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:        
        print(r["err"])
        return None   
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://dlfa.iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if not Additional: 
      raise NameError
    else:
      return Additional  
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None
  
  
# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None
  
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None
  

def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer1():
  try:
    if not Answer1:
      raise NameError 
    else: 
      return Answer1
  except NameError:
    print ("Please answer Question 1")
    return None

def getAnswer2():
  try:
    if not Answer2:
      raise NameError 
    else: 
      return Answer2
  except NameError:
    print ("Please answer Question 2")
    return None
  

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup() 
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


 **Install PySpark**

In [1]:
!pip -q install pyspark

     |████████████████████████████████| 212.4 MB 59 kB/s 
     |████████████████████████████████| 198 kB 53.0 MB/s 


**Creating Spark Session**

Spark session is a combined entry point of a Spark application, which came into implementation from Spark 2.0 (Instead of having various contexts, everything is encapsulated in a Spark session)

In [2]:
# Start spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf  # User Defined Functions
from pyspark.sql.types import StringType
from operator import add
spark = SparkSession.builder.appName('Paired_RDD').getOrCreate()
spark

In [3]:
# Accessing sparkContext from sparkSession instance.
sc = spark.sparkContext

### Transformations

Transformation is a function that produces new RDD from the existing RDDs. It takes RDD as input and produces one or more RDD as output.

Example:- map, filter, flatMap, groupByKey, reduceByKey, etc.

**map():** Map transformation is applied over each and every function of an RDD / Data Frame. The return type is a new RDD or dataframe where the Map function is applied. It is used to apply operations over every element.

1.Perform square of a number by map transformation for the below defined data.

data = [ 1 , 2 , 3 , 4 , 5 , 6 , 7] 

In [18]:
# CODE HERE
#1)Create RDD out of data 
data = [ 1 , 2 , 3 , 4 , 5 , 6 , 7]
numRDD = sc.parallelize(data)
#2)define a function to square elements
def square(x):
  return (x*x)

#3) Apply map to square the number
squareRDD = numRDD.map(square)

print(squareRDD.collect())


[1, 4, 9, 16, 25, 36, 49]


**flatMap():**  It applies function to the elements and returns only a single RDD and it will not return a nested one. The most simple use of flatMap() is to split each input string into words. The key difference between map() and flatMap() is that map() returns only one element, while flatMap() can return a list of elements.


1. Apply the flatMap transformation on the below defined data and check how the data is being transformed.

  data = [ 1 , 2 , 3 , 4 , 5 , 6 , 7] 

In [13]:
# CODE HERE


data = [ 1 , 2 , 3 , 4 , 5 , 6 , 7]
num1RDD = sc.parallelize(data)
#2)define a function to square elements
def square(x):
  return (x*x)

#3) Apply flatMap to square the number
squareRDD1 = num1RDD.flatMap(square)

squareRDD1.collect()



Py4JJavaError: ignored

**Union Operation:** Union is used to append two dataframes with same schema.

1. Use union operation for two RDD's given below:
  
  data1 = [ 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9]

  data2 = [ 2 , 4 , 5 , 7 , 9 ]

In [15]:
# CODE HERE
data1 = [ 1 , 2 , 3 , 4 , 5 , 6 , 7,8,9]
data2 = [ 2 , 4 , 5 , 7 , 9 ]
data1RDD = sc.parallelize(data1)
data2RDD = sc.parallelize(data2)
UnionRDD = data1RDD.union(data2RDD)
UnionRDD.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 4, 5, 7, 9]

2. Use union operator and merge the below given data.

  data  = [ " mango " , " banana " , " apple "]
  
  data2 = [ " orange " , " grapes " , " pineapple " , " mango " , " banana "]

In [16]:
# CODE HERE
data1 = [ " mango " , " banana " , " apple "]

data2 = [ " orange " , " grapes " , " pineapple " , " mango " , " banana "]
data1RDD = sc.parallelize(data1)
data2RDD = sc.parallelize(data2)
UnionRDD = data1RDD.union(data2RDD)
UnionRDD.collect()

[' mango ',
 ' banana ',
 ' apple ',
 ' orange ',
 ' grapes ',
 ' pineapple ',
 ' mango ',
 ' banana ']

**Intersection:-** Using Intersection, we get only the common element of both the RDD's in a new RDD. The key rule of this function is that the two RDDs should be of the same type.

1. For the given below data find the common terms by using the intersection transformation.

  data1 = [1, 10, 2, 3, 4, 5]

  data2 = [1, 6, 2, 3, 7, 8]

In [17]:
# CODE HERE
data1 = [1, 10, 2, 3, 4, 5]
data2 = [1, 6, 2, 3, 7, 8]
data1RDD = sc.parallelize(data1)
data2RDD = sc.parallelize(data2)
commonRDD = data1RDD.intersection(data2RDD)
commonRDD.collect()

[1, 2, 3]

2. For the given below data find the common terms by using the intersection transformation.

  data = [ " mango " , " banana " , " apple " ] 

  data2 = [ " orange " , " grapes " , " pineapple " , " mango " , " banana " ]

In [18]:
# CODE HERE
data1 = [ " mango " , " banana " , " apple " ]
data2 = [ " orange " , " grapes " , " pineapple " , " mango " , " banana " ]
data1RDD = sc.parallelize(data1)
data2RDD = sc.parallelize(data2)
commonRDD = data1RDD.intersection(data2RDD)
commonRDD.collect()

[' mango ', ' banana ']

**filter():** filter() function returns a new RDD containing only the elements that meet a predicate. 

1. Apply filter operation to the data defined below to find the perfect squares.

  data1 = [1, 2, 3, 4, 7, 9, 12, 16, 21, 24, 25, 27, 30, 32 , 36, 44, 47, 49]

In [31]:
# CODE HERE
import numpy as np
def perfect_square(num):
  sqroot = int(np.sqrt(num))
  if(sqroot*sqroot == num):
    return True
  else:
    return False

data = [1, 2, 3, 4, 7, 9, 12, 16, 21, 24, 25, 27, 30, 32 , 36, 44, 47, 49]
dataRDD = sc.parallelize(data)

perfectSquareRDD = dataRDD.filter(perfect_square)
perfectSquareRDD.collect()
#print(perfect_square(100))

[1, 4, 9, 16, 25, 36, 49]

**groupByKey():** When we use groupByKey() on a Paired RDD of (K, V) pairs, the data is shuffled according to the key value K.

1. Apply groupbykey operation to the data defined below.

  data = [ ( " banana " ,  1 ) , ( " apple ", 1 ), ( "carrot " , 1 ) , ( " banana " , 3 ) ]

In [32]:
# CODE HERE
data = [ ( " banana " , 1 ) , ( " apple ", 1 ), ( "carrot " , 1 ) , ( " banana " , 3 ) ]
dataRDD = sc.parallelize(data)

dataRDD.groupByKey().collect()


[(' apple ', <pyspark.resultiterable.ResultIterable at 0x7fda6db35290>),
 (' banana ', <pyspark.resultiterable.ResultIterable at 0x7fda6db35250>),
 ('carrot ', <pyspark.resultiterable.ResultIterable at 0x7fda68f98650>)]

**reduceByKey:** When we use reduceByKey on a Paired RDD (K, V), the pairs on the same machine with the same key are combined, before the data is shuffled.

1. Apply reducebykey operation to the data defined below.

  data = [ ( " banana " , 1), ( " apple " , 1), ( " carrot " , 1), ( " banana " , 3 ) ]

In [36]:
# CODE HERE
data = [ ( " banana " , 1 ) , ( " apple ", 1 ), ( "carrot " , 1 ) , ( " banana " , 3 ) ]
dataRDD = sc.parallelize(data)

def accumulate(acc,num):
  return(acc + num)
  
dataRDD.reduceByKey(accumulate).collect()

[(' apple ', 1), (' banana ', 4), ('carrot ', 1)]

**combineByKey:** It  is very similar to combiner in Hadoop MapReduce programming. Internally, the combineByKey function efficiently combines the values of a PairRDD partition by applying aggregation function. The main objective of combineByKey transformation is transforming any $PairRDD[(K,V)]$ to the $RDD[(K,C)]$ where $C$ is the result of any aggregation of all values under key $K$.

Spark combineByKey function uses following three functions as an argument,

* createCombiner
* mergeValue
* mergeCombiners

1. Find the average of the scores that are scored by each student by using combineByKey transformation. The student marks data is defined below.

      student_rdd = sc.parallelize([
  ("ram", "Maths", 83), ("ram", "Physics", 74), ("ram", "Chemistry", 91), ("ram", "Biology", 82), 
  
  ("suresh", "Maths", 69), ("suresh", "Physics", 62), ("suresh", "Chemistry", 97), ("suresh", "Biology", 80), 
  
  ("ramesh", "Maths", 78), ("ramesh", "Physics", 73), ("ramesh", "Chemistry", 68), ("ramesh", "Biology", 87), 
  
  ("john", "Maths", 87), ("john", "Physics", 93), ("john", "Chemistry", 91), ("john", "Biology", 74), 
  
  ("sita", "Maths", 56), ("sita", "Physics", 65), ("sita", "Chemistry", 71), ("sita", "Biology", 68), 
  
  ("harsha", "Maths", 86), ("harsha", "Physics", 62), ("harsha", "Chemistry", 75), ("harsha", "Biology", 83), 
  
  ("Anu", "Maths", 63), ("Anu", "Physics", 69), ("Anu", "Chemistry", 64),   ("Anu", "Biology", 60)], 3)

In [4]:
# CODE HERE
dataRDD  = sc.parallelize([ ("ram", "Maths", 83), ("ram", "Physics", 74), ("ram", "Chemistry", 91), ("ram", "Biology", 82),

("suresh", "Maths", 69), ("suresh", "Physics", 62), ("suresh", "Chemistry", 97), ("suresh", "Biology", 80),

("ramesh", "Maths", 78), ("ramesh", "Physics", 73), ("ramesh", "Chemistry", 68), ("ramesh", "Biology", 87),

("john", "Maths", 87), ("john", "Physics", 93), ("john", "Chemistry", 91), ("john", "Biology", 74),

("sita", "Maths", 56), ("sita", "Physics", 65), ("sita", "Chemistry", 71), ("sita", "Biology", 68),

("harsha", "Maths", 86), ("harsha", "Physics", 62), ("harsha", "Chemistry", 75), ("harsha", "Biology", 83),

("Anu", "Maths", 63), ("Anu", "Physics", 69), ("Anu", "Chemistry", 64), ("Anu", "Biology", 60)], 3)

def accumulate(acc,num):
  return(acc + num)
  
dataRDD.combineByKey()

Py4JJavaError: ignored

**mapValues:** mapValues function does not change the key. It applies a function to each value of a paired RDD of spark and it retains the partitioning.

1. Find the number of programming languages that a person know in the below defined data by using the mapValues transformation.

  data = [ ( " bharat ", [ " c " , " java " , " python "]),
  
  ( "chandu",[ " c " , " java " , " python " , " scala " ] ) ,
  
  ( "akash", [" c " , " java " , " python " , " scala " , " spark " , " cobal " ] ) ,
  
  ( " deepak " , [ " c " , " java " , "python " ] ) ]

In [ ]:
# CODE HERE

**countByValue:** It count's the number of elements for each value.

1. For the below defined data count the value(Using countByValue action) for the elements.

  data = [1,1,1,2,2,3,3,1,1,2,3,3,2,2,3,3]
    
 

In [ ]:
# CODE HERE

**Cartesian product:** The Cartesian function generates a Cartesian product of two RDDs and returns all the possible combination of pairs. Here, each element of one RDD is paired with each element of another dataset.

1.Find the cartesian product of the given below data.

data1 = [ " spark " , " java " , " python " , " scala " , " c "]

data2 = [ " Hive " , " kafka " , " H-base " ]

In [ ]:
# CODE HERE

2.Implement cartesian product on the dataset defined below

x = { ' A ' ,  2 , ' C '}

y = { ' D ' , 1 }

In [ ]:
# CODE HERE

**distinct:** Return a new RDD containing the distinct elements in this RDD.

1. Find the distinct elements from the below given data.

  data = [ 1 , 1 , 2 , 2 , 3 , 6 , 0 , 5 , 4 , 5 , 6 , 8 , 5 , 3 , 5 , 7 , 4 , 2 , 4 , 6 , 0 ]

In [ ]:
# CODE HERE

2. FInd the distint elements from the below defined data.

  data = [ " c " , " c++ " ,  " c# ", " java " , " python " , " scala " , " c " , " java " , " python " , " scala " , " spark " , " cobal " , " c++ " , " c# " ]

In [ ]:
# CODE HERE

### Please answer the questions below to complete the experiment:




In [8]:
#@title Q.1. What is the function we use when we come across disk spilling problem while handling the data?  { run: "auto", form-width: "500px", display-mode: "form" }
Answer1 = "repartition()" #@param ["","repartition()", "filter()","map()", "All the above"]


In [9]:
#@title Q.2.  Statement: We can save intermediate results only to Disk using Persist.(True or False){ run: "auto", form-width: "500px", display-mode: "form" }
Answer2 = "False" #@param ["","False", "True"]


In [13]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Too Simple, I am wasting time" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [14]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "Large scale dataset can be given and questions can be based on that. Instead of giving datasets of 20-30 samples." #@param {type:"string"}


In [15]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]


In [16]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Somewhat Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [17]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "Didn't use" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [18]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 1180
Date of submission:  25 Jul 2021
Time of submission:  16:16:42
View your submissions: https://dlfa.iisc.talentsprint.com/notebook_submissions
